In [9]:
import requests
import json
import pandas as pd
import feedparser as fp
import newspaper as np

def mettre_a_jour_base_de_donnees(flux_rss_a_lire, database, categories):
    # Liste des nouveaux articles
    new_articles = []

    # Parcourir les flux RSS
    for source_url in flux_rss_a_lire:
        # Déterminer la catégorie du flux
        category = categories[source_url]

        data = fp.parse(source_url)
        
        # Parcourir les articles du flux RSS
        for item in data.entries:
            # Clés renommées pour correspondre à la base de données existante
            article = {
                "title": item.title,
                "date": item.published,  # Renommé de 'published' à 'date'
                "url": item.link,  # Renommé de 'link' à 'url'
                "content": item.description if "description" in item else "",
                "image_link": item.enclosures[0].url if "enclosures" in item and item.enclosures[0].type == "image/jpeg" else "",
                "category": category,  # Ajouté la catégorie
            }
            
            # Vérifier si l'article existe déjà dans la base de données
            if article["url"] not in database:
                article_obj = np.Article(article["url"])
                article_obj.download()
                article_obj.parse()
                
                # Mettre à jour les informations de l'article
                article["content"] += "\n\n" + article_obj.text  # Ajouter le contenu téléchargé
                article["author"] = article_obj.authors  # Ajouté la liste des auteurs
                
                # Ajouter l'article à la liste des nouveaux articles
                new_articles.append((article["url"], article))

    # Mettre à jour la base de données existante avec les nouveaux articles
    for url, article in new_articles:
        database[url] = article

    return database


# Lien vers le fichier tvinfo-sources.json
tvinfo_sources_url = "https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/tvinfo-sources.json"

# Charger la liste des flux RSS depuis le lien tvinfo-sources.json
response_sources = requests.get(tvinfo_sources_url)
sources = json.loads(response_sources.text)

# Liste des flux RSS à parcourir
flux_rss_a_lire = list(sources.values())

# Usage de la fonction avec les catégories
categories = {
    "https://www.francetvinfo.fr/france.rss": "france",
    "https://www.francetvinfo.fr/monde/europe.rss": "europe",  # Corrigé ici
    "https://www.francetvinfo.fr/economie/entreprises.rss": "economie"
}


# Charger la base de données existante depuis le lien francetvinfo.json
francetvinfo_url = "https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/francetvinfo.json"
response_database = requests.get(francetvinfo_url)
database = json.loads(response_database.text)

# Mettre à jour la base de données avec les nouveaux articles
database_mise_a_jour = mettre_a_jour_base_de_donnees(flux_rss_a_lire, database, categories)

# Transformer la base de données mise à jour en DataFrame
df_database_mise_a_jour = pd.DataFrame(database_mise_a_jour.values())

# Ajouter les URLs comme colonne dans le DataFrame
df_database_mise_a_jour['url'] = list(database_mise_a_jour.keys())

# Afficher les premières lignes pour vérifier
df_database_mise_a_jour





,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,NaN,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,NaN,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,NaN,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,NaN,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,NaN,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2152,Leasing social pour les voitures électriques :...,"Mon, 12 Feb 2024 12:54:42 +0100",[Emmanuel Cugny],economie,"Le ministre de l’Industrie et de l’énergie, Ro...",https://www.francetvinfo.fr/pictures/ccdYzaC2l...,NaN,https://www.francetvinfo.fr/replay-radio/le-de...
2153,Comment féminiser les métiers de terrain dans ...,"Sun, 11 Feb 2024 08:47:43 +0100",[Sarah Lemoine],economie,La part des femmes a légèrement progressé dans...,https://www.francetvinfo.fr/pictures/-GnreVvJ2...,NaN,https://www.francetvinfo.fr/replay-radio/c-est...
2154,Entreprises : la crise de l’immobilier pèse su...,"Fri, 09 Feb 2024 09:14:50 +0100",[Emmanuel Cugny],economie,"Les Français ont moins acheté de lave-linges, ...",https://www.francetvinfo.fr/pictures/AnPZT-kFs...,NaN,https://www.francetvinfo.fr/replay-radio/le-br...
2155,"Semaine de quatre jours : ""Il va falloir ident...","Thu, 08 Feb 2024 20:19:53 +0100","[Emmanuel Cugny, Marie Bouny]",economie,"Marie Bouny, docteure en droit et associée au ...",,NaN,https://www.francetvinfo.fr/replay-radio/l-int...


# Mettre à jour la base de données de Git

In [10]:

# Fonction pour récupérer la base de données actuelle depuis GitHub
def telecharger_base_donnees_github(url):
    response = requests.get(url)
    data = response.text.splitlines()  # Séparer chaque ligne du fichier JSON
    df = pd.DataFrame([pd.read_json(line, typ='series') for line in data])
    return df

# Fonction pour concaténer l'ancienne base de données avec les nouveaux articles
def concatener_bases_donnees(df_ancienne, df_nouveaux_articles):
    # Filtrer les nouveaux articles pour ne garder que ceux qui n'existent pas déjà dans l'ancienne base de données
    nouveaux_articles_uniques = df_nouveaux_articles[~df_nouveaux_articles['url'].isin(df_ancienne['url'])]
    # Concaténer les deux DataFrames
    df_mise_a_jour = pd.concat([df_ancienne, nouveaux_articles_uniques], ignore_index=True)
    return df_mise_a_jour

# URL de la base de données sur GitHub
url_github = 'https://raw.githubusercontent.com/diakitegaoussou4996/ENSAI-PROJET-WEBMING/main/Data/francetvinfo_maj.json'

# Charger la base de données actuelle
df_ancienne = telecharger_base_donnees_github(url_github)

# Concaténer l'ancienne base de données avec les nouveaux articles
df_mise_a_jour = concatener_bases_donnees(df_ancienne, df_database_mise_a_jour)

# Sauvegarder la base de données mise à jour localement
df_mise_a_jour.to_json("Data/francetvinfo_maj.json", orient="records", lines=True, force_ascii=False)
df_mise_a_jour.to_csv("Data/francetvinfo_maj.csv", index=False)


In [11]:
df_mise_a_jour

,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,None,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,None,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,None,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,None,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,None,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2295,"Ukraine, deux ans de guerre : les familles de ...","Fri, 23 Feb 2024 07:25:08 +0100","[Lucie Barbarin, Agathe, Batia, Père D'Un Sold...",europe,"En deux ans, le conflit avec la Russie a fait ...",https://www.francetvinfo.fr/pictures/yR72XmRh3...,NaN,https://www.francetvinfo.fr/monde/europe/manif...
2296,DIRECT. Guerre en Ukraine : les Etats-Unis von...,"Fri, 23 Feb 2024 06:33:51 +0100","[Alice Galopin, Fabien Jannic-Cherbonnel]",europe,Une porte-parole du département du Trésor amér...,https://www.francetvinfo.fr/pictures/2YT8ZigRv...,NaN,https://www.francetvinfo.fr/monde/europe/manif...
2297,Guerre en Ukraine : Emmanuel Macron organisera...,"Fri, 23 Feb 2024 05:52:52 +0100",[Franceinfo Avec Afp],europe,Lors de la signature d'un accord bilatéral de ...,https://www.francetvinfo.fr/pictures/WXoAxCijc...,NaN,https://www.francetvinfo.fr/monde/europe/manif...
2298,"Guerre en Ukraine : après deux ans de combats,...","Thu, 22 Feb 2024 22:51:09 +0100",[],europe,"Présente sur le plateau du 20 Heures, jeudi 22...",,NaN,https://www.francetvinfo.fr/monde/europe/manif...


In [12]:
import pandas as pd
# URL du fichier mise à jour chaque jour JSON sur GitHub
url = 'https://raw.githubusercontent.com/diakitegaoussou4996/ENSAI-PROJET-WEBMING/main/Data/francetvinfo_maj.json'
# Charger les données dans un DataFrame
francetvinfo_mise_a_jour = pd.read_json(url, lines=True)
# Afficher les premières lignes du DataFrame
francetvinfo_mise_a_jour

,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,None,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,None,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,None,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,None,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,None,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2276,"Ukraine : deux ans de guerre, et après ?","Thu, 22 Feb 2024 20:25:00 +0100",[France],europe,Le Talk de franceinfo revient sur la guerre en...,https://www.francetvinfo.fr/pictures/VaeOUUUVN...,None,https://www.francetvinfo.fr/monde/europe/manif...
2277,Guerre en Ukraine : 14 millions de personnes o...,"Thu, 22 Feb 2024 19:58:32 +0100",[],europe,L'invasion russe a déclenché la plus grande cr...,https://www.francetvinfo.fr/pictures/HPLKNUQBu...,None,https://www.francetvinfo.fr/monde/europe/manif...
2278,Le Parlement polonais libéralise l'accès à la ...,"Thu, 22 Feb 2024 19:41:51 +0100",[Franceinfo Avec Afp],europe,"En 2017, le précédent gouvernement populiste c...",https://www.francetvinfo.fr/pictures/5J_AFfNgR...,None,https://www.francetvinfo.fr/monde/europe/le-pa...
2279,"Agriculteurs : rumeurs, inquiétudes et fausses...","Thu, 22 Feb 2024 18:20:06 +0100","[Angélique Bouin, France]",europe,"Un projet de permis européen harmonisé, actuel...",https://www.francetvinfo.fr/pictures/w5amfxT4i...,None,https://www.francetvinfo.fr/monde/europe/union...
